In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-test-data/Test_Images/image_22.jpg
/kaggle/input/new-test-data/Test_Images/image_53.jpg
/kaggle/input/new-test-data/Test_Images/image_141.jpg
/kaggle/input/new-test-data/Test_Images/image_69.jpg
/kaggle/input/new-test-data/Test_Images/image_68.jpg
/kaggle/input/new-test-data/Test_Images/image_175.jpg
/kaggle/input/new-test-data/Test_Images/image_166.jpg
/kaggle/input/new-test-data/Test_Images/image_153.jpg
/kaggle/input/new-test-data/Test_Images/image_176.jpg
/kaggle/input/new-test-data/Test_Images/image_51.jpg
/kaggle/input/new-test-data/Test_Images/image_178.jpg
/kaggle/input/new-test-data/Test_Images/image_88.jpg
/kaggle/input/new-test-data/Test_Images/image_132.jpg
/kaggle/input/new-test-data/Test_Images/image_4.jpg
/kaggle/input/new-test-data/Test_Images/image_75.jpg
/kaggle/input/new-test-data/Test_Images/image_124.jpg
/kaggle/input/new-test-data/Test_Images/image_180.jpg
/kaggle/input/new-test-data/Test_Images/image_179.jpg
/kaggle/input/new-test-data/Test_Imag

In [44]:
CUDA_LAUNCH_BLOCKING=1
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image,UnidentifiedImageError
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
#HyperParameters:
learning_rate=0.0001
num_epochs=30
batch_size=128
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = '/kaggle/input/aivsreal/final_data/Data/Train'
real_images_path = os.path.join(data_dir, 'Real')
ai_images_path = os.path.join(data_dir, 'AI')

real_images = [os.path.join(real_images_path, f) for f in os.listdir(real_images_path) if f.endswith(('.jpg', '.png'))]
ai_images = [os.path.join(ai_images_path, f) for f in os.listdir(ai_images_path) if f.endswith(('.jpg', '.png'))]

image_paths = real_images + ai_images
labels = [0] * len(real_images) + [1] * len(ai_images) 


train_paths, valid_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)




class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        
        try:
            #
            image = Image.open(img_path).convert("RGB")
        except UnidentifiedImageError:
            print(f"Warning: Unidentified or corrupted image at {img_path}.")
            
            
           
            image = Image.new("RGB", (224, 224), color=(0, 0, 0))
        
        
        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label



In [45]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),#flips the image 
    transforms.RandomRotation(15),#
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),#add some charecterstics,
    transforms.ToTensor(),
     
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CustomImageDataset(train_paths, train_labels, transform=transform)
valid_dataset = CustomImageDataset(valid_paths, valid_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [46]:
example=iter(train_loader)
samples,labels=next(example)
print(samples.shape,labels.shape)
print(type(labels))
labels = labels.to(torch.long)

torch.Size([128, 3, 224, 224]) torch.Size([128])
<class 'torch.Tensor'>


In [47]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5, 5))
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout1 = nn.Dropout(0.2)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=256, kernel_size=(3, 3))
        self.batch_norm2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout2 = nn.Dropout(0.3)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3))
        self.batch_norm3 = nn.BatchNorm2d(512)
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout3 = nn.Dropout(0.4)
        
        # Fourth convolutional block
        self.conv4 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3))
        self.batch_norm4 = nn.BatchNorm2d(1024)
        self.pool4 = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout4 = nn.Dropout(0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(1024 * 12 * 12, 2048)  
        self.dropout5 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(2048, 2)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = F.relu(self.conv3(x))
        x = self.batch_norm3(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = F.relu(self.conv4(x))
        x = self.batch_norm4(x)
        x = self.pool4(x)
        x = self.dropout4(x)
        
        x = torch.flatten(x, 1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout5(x)
        
        x = (self.fc2(x))
        return x

In [48]:

  # Adjust weights for imbalance
loss= nn.CrossEntropyLoss()


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.1)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0.0  # Initialize loss accumulator for the epoch


    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        l = loss(outputs, labels)

        # Backward pass and optimization
        l.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Accumulate loss for epoch
        epoch_loss += l.item()

    # Step the scheduler after the epoch
    scheduler.step()

    # Calculate and print the average epoch loss
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1} Completed. Average Loss: {avg_epoch_loss:.6f}, LR: {scheduler.get_last_lr()[0]:.6f}")
   
    

Epoch 1 Completed. Average Loss: 0.261308, LR: 0.000100
Epoch 2 Completed. Average Loss: 0.261633, LR: 0.000100
Epoch 3 Completed. Average Loss: 0.272624, LR: 0.000100
Epoch 4 Completed. Average Loss: 0.267777, LR: 0.000100
Epoch 5 Completed. Average Loss: 0.277038, LR: 0.000010
Epoch 6 Completed. Average Loss: 0.255171, LR: 0.000010
Epoch 7 Completed. Average Loss: 0.264899, LR: 0.000010
Epoch 8 Completed. Average Loss: 0.281764, LR: 0.000010
Epoch 9 Completed. Average Loss: 0.253835, LR: 0.000010
Epoch 10 Completed. Average Loss: 0.280265, LR: 0.000001
Epoch 11 Completed. Average Loss: 0.262073, LR: 0.000001
Epoch 12 Completed. Average Loss: 0.272935, LR: 0.000001
Epoch 13 Completed. Average Loss: 0.263299, LR: 0.000001
Epoch 14 Completed. Average Loss: 0.263570, LR: 0.000001
Epoch 15 Completed. Average Loss: 0.275418, LR: 0.000000
Epoch 16 Completed. Average Loss: 0.260155, LR: 0.000000
Epoch 17 Completed. Average Loss: 0.259548, LR: 0.000000
Epoch 18 Completed. Average Loss: 0.2677

In [49]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for (images,labels) in (valid_loader):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    _, predicted_cls = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += (predicted_cls == labels).sum().item()

accuracy=correct/total
print("Accuracy:"+f"{accuracy*100:.3f}")  

Accuracy:86.022


In [50]:
test_images_path ='/kaggle/input/new-test-data/Test_Images'

test_images = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png'))]


transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [51]:
class TestImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

       
        self.image_paths = [path for path in self.image_paths if self.is_valid_image(path)]

    def is_valid_image(self, img_path):
        try:
         
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(img_path)

test_dataset = TestImageDataset(test_images, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()  
predictions = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)  # Move images to the correct device
        outputs = model(images)  # Get model predictions
        
        # Since the model already uses BCELoss, outputs are probabilities (between 0 and 1)
        _,predicted_cls_test =torch.max(outputs, dim=1)# Apply threshold to get binary classes
        
        # Append predictions with corresponding filenames
        for filename, label in zip(filenames, predicted_cls_test):
            label_str = 'AI' if label.item() == 1 else 'Real'  # Assign class names
            predictions.append((filename, label_str))  # Store the prediction


In [53]:
import pandas as pd

submission = pd.DataFrame(predictions, columns=['Id', 'Label'])



submission.to_csv('submission_f11.csv', index=False)

In [ ]:
submission.to_csv('/kaggle/working/submission22.csv', index=False)
